# Importing the necessary libraries

In [3]:
import functions

# 1. Hashing

## Hash Function

We first created a hash function capable of converting every single string into a unique value to avoid possible collisions. The dataset used contains over 560 thousand lines and consequently over half a million strings.To avoid collisions with over half a million strings we have set the maximum number of bits to 32 and consequently a modulo equal to 2<sup>32</sup>.

In [7]:
def hash(s, max_n_bit_hash_value=32):
    max_value = 1 << max_n_bit_hash_value # Define module equal to two to the thirty-second
    l = len(s) # len string
    h = 0
    for i in range(l): # For loop between 0 to len string
        h += s[i] ** (i+1) #For each character we increase "h". We compute for each character this power equation
        h %= max_value #After the power eqaution we divided the value of h by the module
    return h

## HyperLogLog structure

Once we have created the hash function valid for each string of our dataset. We have created the HyperLogLog class divided into four functions.
The *first function* represents an initialization for all the necessary variables in the HyperLogLog structure.
In the *second function* we apply all the variables set on the considered string.
Finally in the *last function* we return the cardinality value for the entire dataset.

The HyperLogLog class returns also the error against our filter.

In [13]:
import numpy as np
from math import sqrt, log

In [12]:
class HyperLogLog:
    def __init__(self, b=10, max_n_bit_hash_value=32):
        self.max_n_bit = max_n_bit_hash_value #We set the max number of bit for the hash value obtained before
        self.b = b 
        self.m = 2**b 
        self.M = np.full(self.m, 0) #An array with size equal to m initialized with all values equal to zero.
        self.a_m = 0.7213 / (1 + 1.079/self.m) #This value was obtained from a publication reported in our references. It is a value that depends only on m
        print("Error Filter:", 1.04/sqrt(self.m), '%')
    
    def add(self, s):
        x = hash(s, max_n_bit_hash_value=self.max_n_bit) #We apply our hash functon for each string that we put in input
        j = x >> (self.max_n_bit - self.b) #We define j as the value x obtained from the hash function divided by 2 raised to the difference between the maximum number of bits set for the hash function and the variable b chosen.
        w = x & 2**(self.max_n_bit - self.b) - 1 
        # w = x & (2**(self.max_n_bit - self.b) - 1) # possibile nuova versione, da verificare le precedenze degli operatori
        self.M[j] = max(self.M[j], self.rho(w, max_length=self.max_n_bit - self.b)) #For each element in array M we upload this with the max value between M[j] and value obtain with rho function.
    
    def rho(self, n, max_length): #?
        p = len(bin(n)[2:]) # Most significant bit in n
        return max_length - p + 1
    
    def card(self):
        Z = 1/np.sum(2.**-self.M) #It is equal to the sum of 2 raised by the opposite of each element belonging to the array M
        E = self.a_m * Z * self.m ** 2 #Cardinality defined by multiplication between the calculated variable Z and the two initialized variables (a_m and m)
        if E < 5.*self.m/2.:
            V = (self.M==0).sum() #Equal to the number of element in array M equal to 0
            if V==0: #If there isn't any element in array M equal to 0
                return E 
            else:
                return self.m*log(self.m*1./V) #The new cardinality is equal to the product of m (2 ** b) and the logarithm
        return E #Return Cardinality estimate with relative error

### Evaluation

Once we realized our Hash function and the HyperLogLog Structure, we took our dataset and carried out various tests to understand the best result that would minimize the Error Filter and increase the cardinality of the dataset.

There are two variables that affect the filter error and cardinality: the number of bit (b) and the maximum number of bits for the hash values.

In [15]:
hll = HyperLogLog(10, 24)

f = open('hash.txt', 'rb')

i=0
for line in f:
    line = line[:-1]
    hll.add(line)

f.close()

hll.card()

Error Filter: 0.0325 %


KeyboardInterrupt: 

In [ ]:
hll = HyperLogLog(12, 24)

f = open('hash.txt', 'rb')

i=0
for line in f:
    line = line[:-1]
    hll.add(line)

f.close()

hll.card()

In [ ]:
hll = HyperLogLog(12, 32)

f = open('hash.txt', 'rb')

i=0
for line in f:
    line = line[:-1]
    hll.add(line)

f.close()

hll.card()

In [ ]:
hll = HyperLogLog(16, 32)

f = open('hash.txt', 'rb')

i=0
for line in f:
    line = line[:-1]
    hll.add(line)

f.close()

hll.card()

Analyzing the various tests carried out, the filter error decreases as the number of bits obtained increases and this is normal since we know that our error filter is inversely proportional to the variable m which in turn is equal to the power of 2 raised to the number of bit.

For a number of bits equal to 16 we obtain an error filter equal to 0.004%.

Considering both cardinality and error filter we get the best result when we apply to the HyperLogLog Structure the number of bits equal to 16 and maximum number of bits equal to 32

# 2. Clustering

Our goal in this part of Homework is to define a KMeans Clustering algorithm from scratch considering the proposed Amazon Food dataset as the initial dataset. To do this it is necessary to prepare the dataset for Clustering. Preparation is strongly influenced by our choices, through the decision of which variables we are going to consider for the Clustering.

We choose "ProductId" and "Text" as columns of the dataset. By choosing these two columns it is necessary to represent the data (column Text) with TF-IDF to create a matrix where for each word we carry the tf-idf associated with each document.

## Pre-Processing dataset

### Visualization Dataset

In [5]:
import pandas as pd
from tqdm import tqdm

In [6]:
dataset = pd.read_csv("/users/antoniozappia/Desktop/Reviews.csv")

In [7]:
dataset

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


### Drop and save dataset

In this part we obtain a copy of the initial dataset that we call "new_dataset".

Using this new_dataset, we are going to drop all the columns that will not be useful for our analysis and so we are going to retain only the following columns:
- ProductId
- Text

In [18]:
new_dataset = dataset.copy()

In [19]:
new_dataset = new_dataset.drop(["Id","UserId", "ProfileName", "HelpfulnessNumerator", "HelpfulnessDenominator", "Score", "Summary", "Time"], axis =1)

In [20]:
new_dataset

,ProductId,Text
0,B001E4KFG0,I have bought several of the Vitality canned d...
1,B00813GRG4,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,This is a confection that has been around a fe...
3,B000UA0QIQ,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,Great taffy at a great price. There was a wid...
...,...,...
568449,B001EO7N10,Great for sesame chicken..this is a good if no...
568450,B003S1WTCU,I'm disappointed with the flavor. The chocolat...
568451,B004I613EE,"These stars are small, so you can give 10-15 o..."
568452,B004I613EE,These are the BEST treats for training and rew...


### Cleaning Text Columns

We are now going to clean the Text Columns using the nltk Library.

For the **clean_text** function, we first broke each signal string into tokens with the tokenizer. After this, we used the stopwords, the lemmatizer and the pos_tag from the nltk library.

With the lemmatizer we got the roots of each single word while with the pos_tag we decided to consider only the words belonging to the category adjectives, adverbs, nouns and verbs. Finally we got the modified string for each line.

In [13]:
new_dataset["Text"] = new_dataset["Text"].apply(lambda text: (functions.clean_text(text)))

In [14]:
new_dataset

,index,ProductId,Text
0,0,B001E4KFG0,buy several vitality can dog food products fin...
1,1,B00813GRG4,product arrive label jumbo salt peanuts peanut...
2,2,B000LQOCH0,confection centuries light pillowy citrus gela...
3,3,B000UA0QIQ,look secret ingredient robitussin believe find...
4,4,B006K2ZZ7K,great taffy great price wide assortment yummy ...
...,...,...,...
568449,568449,B001EO7N10,great sesame chicken good better resturants ea...
568450,568450,B003S1WTCU,disappoint flavor chocolate note especially we...
568451,568451,B004I613EE,star small give train session try train dog ce...
568452,568452,B004I613EE,best treat train reward dog good groom lower c...


### Save new Dataset

Then we saved the dataset inside a csv file.

Before doing this we'll first check if any row in the dataset contains a null field in the Text column. This is necessary as we have to delete lines with null Text columns for the TFIDF Vectorizer.

In [21]:
functions.save_dataset(new_dataset)

In [22]:
new_dataset = pd.read_csv("New_database.csv")

Once we have this new dataset, we are going to join all the texts corresponding to each single ProductId.

In [26]:
new_dataset = new_dataset.groupby('ProductId', as_index = False).agg({'Text': ' '.join})

,ProductId,Text
0,B001E4KFG0,I have bought several of the Vitality canned d...
1,B00813GRG4,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,This is a confection that has been around a fe...
3,B000UA0QIQ,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,Great taffy at a great price. There was a wid...
...,...,...
568449,B001EO7N10,Great for sesame chicken..this is a good if no...
568450,B003S1WTCU,I'm disappointed with the flavor. The chocolat...
568451,B004I613EE,"These stars are small, so you can give 10-15 o..."
568452,B004I613EE,These are the BEST treats for training and rew...


## Tf-Idf Vectorizer 

Once we got clean texts for each row of the dataset we applied from the Scikit Learn library the TFIDFVectorizer. Thanks to this we got a matrix that contains in the rows all the words in the dataset (in numeric format) and as columns all the documents belonging to the dataset.

For each word / document, if that word is in the considered document, we report the TFIDF value.

In [25]:
tfidf = functions.tfidf_vectorizer(new_dataset)

NameError: name 'new_dataset' is not defined

### Trade OFF Number Components - Variance

Once we have applied the TFIDF method, we have realized that tens of thousands of words compose our vocabulary.

For this reason we decided to reduce the dimensionality of the dataset using the SVDMethod.

Before doing this we need to find the correct number of components that we are going to consider for the SVD Method. We are going to consider only those components which will return an aggregate variance of  at least 60.

To do this we solved this TRADEOFF between Components and Variance both analytically and graphically.

In [30]:
best_component = functions.best_compenents(tfidf,700,1000,60)

In [31]:
best_component2 = functions.best_compenents2 (tfidf,1000,50)

In [ ]:
functions.plot_components(best_component2)

## SVD Method

Once we got the useful number of components, we were able to apply the SVDMethod.

In [ ]:
new_matrix = functions.SVDMethod(tfidf,best_component)

## KMeans

We built our KMeans algorithm by following this procedure:

### K-Means Clustering

1. Choose the number of clusters(K) and obtain the data points
2. Place the centroids c_1, c_2, ..... c_k randomly
3. Repeat steps 4 and 5 until convergence or until the end of a fixed number of iterations
4. for each data point x_i:
     - find the nearest centroid(c_1, c_2 .. c_k) 
     - assign the point to that cluster 
     
     
5. for each cluster j = 1..k
     - new centroid = mean of all points assigned to that cluster


6. End

Before continuing with the Clustering procedure on our algorithm, we searched for the best value of K (number of clusters).

We applied the elbow method to the KMeans algorithm obtained from the Scikit-Learn library.

To reduce the computation time in the search for this optimal K value, we have created a list where we selected for which values we should compute k. The list contains integer values that increase more and more up to a maximum value we set.

The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow (or something that is assimilated to it) of the curve as the number of clusters to use.

From the plot we obtained it is not possible to visualize a real elbow beyond which there is a radical flattening of the curve. Variations beyond which the trend of the curve begins to be almost linear can be found in a value of K equal to 20.

We are now going to apply our KMeans algorithm and we will then compare it with the KMeans implemented on the Scikit-Learn library.

We decided to print two plots for our algorithm.

The first represents the trend of the inertia as the number of iterations varies. From this plot we can notice that in the first 10 iterations the inertia tends to drop more heavily and then stabilize with very slight variations.

In the second plot we represented the changes in each single iteration. Also in this case there are important changes in the first operations that converges to zero after few iterations.

## Analyse the obtained clusters

Before answering what can be considered as our final considerations on the clusters we have obtained, it is necessary to do certain preliminary steps. Let's see them!

With the **complete_dataset** function we joined two different datasets: the initial dataset coming from the csv file Reviews and a second dataset which had only 3 columns - ProductId, the cleaned text (as we have seen beforehand) and the third columns tells us the cluster to which each ProductId is associated.

Furthermore, on this last dataset we have obtained, we groupedBy on the ProductId and applied also a join on the ProductId column.

Finally, we have saved the dataset in a csv file. This was done in order not to have to compute all this preprocessing on the data everytime and so beeing more efficient. On this file we are going to work.


In [ ]:
functions.complete_dataset(new_dataset)

In [ ]:
complete_dataset = pd.read_csv("Complete_dataset.csv")

In [ ]:
complete_dataset.head()

### 2.3.1 Identify the kind of products in the cluster

### 2.3.2 Provide the number of product in each cluster

To answer this question, we first applied a GroupBy on the Cluster column. Once we did this, we counted the number of rows for each cluster which exactly matches the number of Products for each cluster.

### 2.3.3 Compute the reviews' score distribution in each cluster

In order to calculate the distribution of the review score, we performed a for loop on the total number of clusters K. For each cluster we searched for the associated scores in the final dataset and we represented everything as a histogramm.

### 2.3.4 Get the number of unique users writing reviews in each cluster

To search for the number of unique users writing reviews in each cluster, we carried out a GroupBy on the Cluster column and counted the unique UserId'sthrough the nunique command.

# 3. Integer Sort

In [21]:
def integer_sort(A, n):

    #Find the maximum and the minimum given an array A has complexity O(n)
    s = min(A)
    b = max(A)
    
    #Allocate new array has time complexity O(1)
    r = int(b - s)
    pos_arr = new_array(value=0, dim=r)
    res = new_array(dim = n)
    idx = 0
    
    #This chunk loops n times over the array A so its comlexity is O(n)
    for i in range(1,n):
        p = A[i]
        pos_arr[p - s] += 1
        
    #This chunk loops r times over the array A so its comlexity is O(r) = O(b - s)
    for i in range(1,r):
        if pos_arr[i] != 0:
            for j in range(1,pos_arr[i]):
                res[idx] = i + s
                idx += 1
    return res

### Time Complexity


The algorithm complexity is $O(n + r) = O(n +(b - s))$

# References

 * For HyperLogLog structure: http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf